## Inconsistent Data Entries


Dataset: The dataset contains detailed information of 475 suicide bombing attacks in Pakistan that killed an estimated 6,982 and injured 17,624 people.

In [1]:
import pandas as pd
import numpy as np
import fuzzywuzzy
from fuzzywuzzy import process
import chardet


C:\Users\Mega\AppData\Local\Programs\Python\Python39\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
data = pd.read_csv("./PakistanSuicideAttacks.csv")

In [3]:
data.head(3)

,S#,Date,Islamic Date,Blast Day Type,Holiday Type,Time,City,Latitude,Longitude,Province,...,Targeted Sect if any,Killed Min,Killed Max,Injured Min,Injured Max,No. of Suicide Blasts,Explosive Weight (max),Hospital Names,Temperature(C),Temperature(F)
0,1,Sunday-November 19-1995,25 Jumaada al-THaany 1416 A.H,Holiday,Weekend,NaN,Islamabad,33.7180,73.0718,Capital,...,None,14.0,15.0,NaN,60,2.0,NaN,NaN,15.835,60.503
1,2,Monday-November 6-2000,10 SHa`baan 1421 A.H,Working Day,NaN,NaN,Karachi,24.9918,66.9911,Sindh,...,None,NaN,3.0,NaN,3,1.0,NaN,NaN,23.770,74.786
2,3,Wednesday-May 8-2002,25 safar 1423 A.H,Working Day,NaN,7:45 AM,Karachi,24.9918,66.9911,Sindh,...,Christian,13.0,15.0,20.0,40,1.0,2.5 Kg,1.Jinnah Postgraduate Medical Center 2. Civil ...,31.460,88.628


In [4]:
data.columns

Index(['S#', 'Date', 'Islamic Date', 'Blast Day Type', 'Holiday Type', 'Time',
       'City', 'Latitude', 'Longitude', 'Province', 'Location',
       'Location Category', 'Location Sensitivity', 'Open/Closed Space',
       'Influencing Event/Event', 'Target Type', 'Targeted Sect if any',
       'Killed Min', 'Killed Max', 'Injured Min', 'Injured Max',
       'No. of Suicide Blasts', 'Explosive Weight (max)', 'Hospital Names',
       'Temperature(C)', 'Temperature(F)'],
      dtype='object')

#### Take a look at the "City" column

In [5]:
data.City.unique()

array(['Islamabad', 'Karachi', 'Karachi ', 'Quetta', 'Rawalpindi',
       'North waziristan', 'Kohat', 'Attock ', 'Sialkot', 'Lahore ',
       'Swat', 'Hangu', 'Bannu', 'North Waziristan', 'Lasbela',
       'Malakand ', 'Peshawar', 'Peshawar ', 'D.I Khan ', 'Lakki Marwat',
       'Tank', 'Islamabad ', 'Quetta ', 'Gujrat', 'Charsadda', 'Tank ',
       'Swat ', 'D.I Khan', 'Kohat ', 'Kuram Agency ', 'Shangla ',
       'Bajaur Agency', 'South Waziristan', 'Haripur', 'bannu',
       'karachi ', 'Sargodha', 'Nowshehra', 'Charsadda ',
       'Mohmand agency', 'Dara Adam Khel', 'Khyber Agency', 'Mardan',
       'Bhakkar ', 'Orakzai Agency', 'Mohmand Agency', 'Buner',
       'D.G Khan ', 'Pishin', 'Chakwal ', 'Lahore', 'Upper Dir',
       'Muzaffarabad', 'swat', 'Totalai', 'Multan', 'D.G Khan',
       'Lower Dir', 'Lakki marwat', 'Sudhanoti', 'Khyber Agency ',
       'peshawar', 'Poonch', 'Mansehra', 'Karak', 'Mohmand Agency ',
       'South waziristan', 'lakki marwat', 'Swabi ', 'Dara Adam kh

### You would see some inconsistent entries at :

- 'Karachi'  and  'Karachi '  (excess space)
-  'Swat '   and  'swat'   (upper and lower case problem)
and more ..

In [6]:
# convert to lower case
data['City'] = data['City'].str.lower()
# remove trailing white spaces
data['City'] = data['City'].str.strip()

In [7]:
data["City"].unique()

array(['islamabad', 'karachi', 'quetta', 'rawalpindi', 'north waziristan',
       'kohat', 'attock', 'sialkot', 'lahore', 'swat', 'hangu', 'bannu',
       'lasbela', 'malakand', 'peshawar', 'd.i khan', 'lakki marwat',
       'tank', 'gujrat', 'charsadda', 'kuram agency', 'shangla',
       'bajaur agency', 'south waziristan', 'haripur', 'sargodha',
       'nowshehra', 'mohmand agency', 'dara adam khel', 'khyber agency',
       'mardan', 'bhakkar', 'orakzai agency', 'buner', 'd.g khan',
       'pishin', 'chakwal', 'upper dir', 'muzaffarabad', 'totalai',
       'multan', 'lower dir', 'sudhanoti', 'poonch', 'mansehra', 'karak',
       'swabi', 'shikarpur', 'sukkur', 'chaman', 'd. i khan', 'khanewal',
       'fateh jang', 'taftan', 'tirah valley', 'wagah', 'zhob',
       'kurram agency', 'taunsa', 'jacobabad', 'shabqadar-charsadda',
       'khuzdar', 'ghallanai, mohmand agency', 'hayatabad',
       'mosal kor, mohmand agency', 'sehwan town',
       'tangi, charsadda district'], dtype=object

####  More difficult inconsistencies :
 - 'mosal kor, mohmand agency'   and    'mohmand agency'
  
### Let's use fuzzywuzzy to find out which strings are closet to each other !
Fuzzy matching: The process of automatically finding text strings that are very similar to the target string, The closer the ratio is to 100, the smaller the edit distance between the two strings.


In [8]:
cities = data.City.unique()
print("Matches for : ", cities[0])
matches = fuzzywuzzy.process.extract('islamabad', cities, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

Matches for :  islamabad


[('islamabad', 100),
 ('jacobabad', 56),
 ('hayatabad', 56),
 ('lakki marwat', 48),
 ('muzaffarabad', 48),
 ('malakand', 47),
 ('swabi', 43),
 ('shabqadar-charsadda', 43),
 ('mardan', 40),
 ('chaman', 40)]

seems like 'islamabad' is good , no similar matches from the human point of view 

- Let's check other city name 


In [9]:
matches = fuzzywuzzy.process.extract('mohmand agency', cities, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

[('mohmand agency', 100),
 ('ghallanai, mohmand agency', 74),
 ('mosal kor, mohmand agency', 74),
 ('orakzai agency', 64),
 ('kuram agency', 62),
 ('bajaur agency', 59),
 ('khyber agency', 59),
 ('kurram agency', 59),
 ('sargodha', 45),
 ('d.g khan', 45)]

Matches > 70 should be replaced 👍

In [10]:
# we will write a function to replace them

def replace_matches_in_column(df, column, string_to_match, min_ratio = 70):
    strings = df[column].unique() # unique names    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings,limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio] # only get matches with a ratio > 70
    print("Close Match " , close_matches)
    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)
    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

In [11]:
replace_matches_in_column(df=data, column='City', string_to_match="mohmand agency")


Close Match  ['mohmand agency', 'ghallanai, mohmand agency', 'mosal kor, mohmand agency']
All done!


In [12]:
data.City.unique()

array(['islamabad', 'karachi', 'quetta', 'rawalpindi', 'north waziristan',
       'kohat', 'attock', 'sialkot', 'lahore', 'swat', 'hangu', 'bannu',
       'lasbela', 'malakand', 'peshawar', 'd.i khan', 'lakki marwat',
       'tank', 'gujrat', 'charsadda', 'kuram agency', 'shangla',
       'bajaur agency', 'south waziristan', 'haripur', 'sargodha',
       'nowshehra', 'mohmand agency', 'dara adam khel', 'khyber agency',
       'mardan', 'bhakkar', 'orakzai agency', 'buner', 'd.g khan',
       'pishin', 'chakwal', 'upper dir', 'muzaffarabad', 'totalai',
       'multan', 'lower dir', 'sudhanoti', 'poonch', 'mansehra', 'karak',
       'swabi', 'shikarpur', 'sukkur', 'chaman', 'd. i khan', 'khanewal',
       'fateh jang', 'taftan', 'tirah valley', 'wagah', 'zhob',
       'kurram agency', 'taunsa', 'jacobabad', 'shabqadar-charsadda',
       'khuzdar', 'hayatabad', 'sehwan town', 'tangi, charsadda district'],
      dtype=object)

In [13]:
matches = fuzzywuzzy.process.extract('mohmand agency', data['City'].unique(),limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches  #only one specific match for 'mohamed agency'

[('mohmand agency', 100),
 ('orakzai agency', 64),
 ('kuram agency', 62),
 ('bajaur agency', 59),
 ('khyber agency', 59),
 ('kurram agency', 59),
 ('sargodha', 45),
 ('d.g khan', 45),
 ('jacobabad', 43),
 ('fateh jang', 42)]

## All correct and our list is clean !